In [4]:
import requests
import datetime

def populate_list(number_days_window):
    populated_list = []
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live')
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live', verify=False)
    all_days = requests.get('https://api.covid19api.com/total/country/uk')
    print(all_days.json()[0]['Country'])
    first=True
    for day in all_days.json():
        date_time_obj = datetime.datetime.strptime(day['Date'][:10], '%Y-%m-%d')
        if first:
            ordinal_first = date_time_obj.toordinal()
        n = date_time_obj.toordinal() - ordinal_first
        delta = 0
        fatality_rate = 0
        if (day['Confirmed'] < 1000):
            continue
        if n>number_days_window:
            # delta mean
            delta = (day['Confirmed'] - populated_list[n-number_days_window][2]) / number_days_window
        first=False
        if day['Confirmed'] > 0:
            fatality_rate = day['Deaths']/day['Confirmed']
        populated_list.append((n, day['Active'], day['Confirmed'], delta, fatality_rate, date_time_obj))
    return populated_list

In [5]:
import pandas as pd
import numpy as np

# Si fatality rate sube, se esta subtesteando, hay mas casos de lo que se cree.
# jump days cada cuantos dias sacas una foto (mirando pa atras)

def process_means(number_days_window, jump_days, total_jumps):
    my_populated_list = populate_list(number_days_window)
    means_list = []
    fatality_rate = []
    active_cases = []
    total_lapse_day = jump_days * total_jumps
    df = pd.DataFrame (my_populated_list,columns=['day-number', 'active', 'cases-count','delta', 'fatality_rate','date'])
    
    for n in range(total_lapse_day,0,-jump_days):
        try:
            mean = round(df[-(n):-(n-(jump_days))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days))]['active'].mean())
        except:
            mean = round(df[-(n):-(n-(jump_days-1))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days-1))]['active'].mean())
        means_list.append(mean)
        fatality_rate.append(df[-n:-(n-1)]['fatality_rate'].mean())
        active_cases.append(active)
    return means_list, fatality_rate, active_cases



In [6]:
import statistics
total_jumps = 3
a = range(total_jumps)

#print(process_means(30,14,total_jumps))

#means_list, fatality_rate, active_cases = process_means(30,14,total_jumps)
from scipy.stats import linregress
for params in [process_means(30,14,total_jumps), process_means(14,14,total_jumps), process_means(10,14,total_jumps), process_means(7,14,total_jumps), process_means(5,14,total_jumps)]:
    means_list, fatality_rate, active_cases = params
    for result in [means_list, fatality_rate, active_cases]:
        print(result)
        print(linregress(a, result))
        print(statistics.pstdev(result), statistics.stdev(result),statistics.pvariance(result),statistics.variance(result))
    print ("...----...")

United Kingdom
United Kingdom
United Kingdom
United Kingdom
United Kingdom
[18755, 20822, 35225]
LinregressResult(slope=8235.0, intercept=16699.0, rvalue=0.9178564114833652, pvalue=0.2598367911430702, stderr=3561.0964603616117)
7325.600453205184 8971.99158492695 53664422 80496633
[0.03594661030245866, 0.03419855571584626, 0.029774994037378873]
LinregressResult(slope=-0.003085808132539893, intercept=0.03639252815110116, rvalue=-0.9700758668566332, pvalue=0.1561330671838627, stderr=0.0007723523698506153)
0.002597272930602815 0.0031809967013600006 6.745826676042136e-06 1.0118740014063205e-05
[1705878, 2067472, 2701603]
LinregressResult(slope=497862.5, intercept=1660455.1666666665, rvalue=0.9877431560615506, pvalue=0.09977667774016749, stderr=78674.65515706655)
411547.3000521595 504040.4450739378 169371180180.22223 254056770270.3333
...----...
[15870, 26339, 47382]
LinregressResult(slope=15756.0, intercept=14107.666666666668, rvalue=0.9817460954197347, pvalue=0.12182484336219512, stderr=30

# Algunas reglas

Si fatality_rate > fatality_rate_mean:
    Yellow Warning Casos pueden ser mayor de lo reflejado pues pueden que estén subtesteando.

Si fatality_rate > fatality_rate_mean y active_cases y means_list en crecimiento:

    Rojo Warning 

means_list, fatality_rate, active_cases


In [65]:
from random import choice
from experta import *


class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])


engine1 = RobotCrossStreet()
engine1.reset()
engine1.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine1.run()


Don't walk
